In [6]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from datetime import datetime
import time

# ----------------------------------
# 날짜 입력받기
#   -> 입력하지 않으면 오늘날짜 반환
#   -> 잘못된 형식이면 None 반환
# ----------------------------------
def date_input():

    # 날짜 입력받기
    date_str = input("날짜:(형식:20200105)")
    
    # 날짜를 입력하지 않으면 오늘 날짜 가져오기
    if date_str=="":
        date_str = datetime.now().strftime('%Y%m%d')
    
    # 날짜 형식 변환 (YYYYmmdd --> YYYY-mm-dd)
    try:
        date_str = datetime.strptime(date_str, '%Y%m%d').strftime('%Y-%m-%d')
    except:
        print("날짜 형식이 올바르지 않습니다.")
        date_str = None
    
    return date_str

date = date_input()

print(f'입력한 날짜 : {date}')


입력한 날짜 : 2024-11-02


In [7]:
page = 1
data = []

while True:
    time.sleep(1)
    
    # ----------------------------------
    # 웹페이지 요청하여 응답객체 받기
    # ----------------------------------
    url = f'https://finance.naver.com/news/mainnews.naver?date={date}&page={page}'
    response = requests.get(url)
        
    # ----------------------------------
    # 응답받은 웹페이지 파싱하여 BeautifulSoup 객체 생성
    # ----------------------------------
    soup = BeautifulSoup(response.text, 'html.parser')

    # ----------------------------------
    # article 리스트 추출
    # ----------------------------------   
    articles = soup.select(".block1")

    # ----------------------------------
    # article 리스트에서 요소의 텍스트, 속성 추출
    # ----------------------------------   
    
    for article in articles:
        title = article.select_one(".articleSubject>a").text
        summary = article.select_one(".articleSummary").contents[0].text.strip()
        press = article.select_one(".press").text.strip()
        wdate = article.select_one(".wdate").text.strip()
        link = article.select_one(".articleSubject>a").attrs['href']
        article_id = link.split('=')[1].split('&')[0]
        office_id = link.split('office_id=')[1].split('&')[0]
        link = f'https://n.news.naver.com/mnews/article/{office_id}/{article_id}'
        
        data.append({"title":title, "summary":summary, "press":press, "wdate":wdate, "link":link})


    # ----------------------------------
    # 페이지 이동
    # ----------------------------------
    if soup.select_one(".pgRR"):
        page += 1
        print(f'=== {page}page ===')
    else:
        break

=== 2page ===
=== 3page ===
=== 4page ===


In [8]:
df = pd.DataFrame(data)
df

,title,summary,press,wdate,link
0,"메타, 만져서 느끼는 ‘촉각 로봇’ 개발…휴머노이드 발전 촉진할 듯",페이스북과 인스타그램 등을 운영하는 메타가 AI 서비스를 넘어 로봇 개발에까지 나섰...,MBN,2024-11-02 21:44:16,https://n.news.naver.com/mnews/article/057/000...
1,'어행'을 떠나요…최대 반값에 사는 수산대축제,서울 노량진수산시장에선 주말 이틀 일정으로 수산대축제가 열렸는데요. 제철 수산물을 ...,MBN,2024-11-02 20:13:08,https://n.news.naver.com/mnews/article/057/000...
2,"트럼프 vs 해리스…다음주 美대선, 누가 대통령 돼야 내 주식 오르나",올해 하반기 전 세계 금융시장의 최대 이벤트로 꼽히는 미국 대통령 선거가 내주로 다...,매일경제,2024-11-02 18:03:10,https://n.news.naver.com/mnews/article/009/000...
3,中 내주 재정정책 승인 전망…WSJ “회의론 더 크다”,중국 중앙정부의 경기 부양 정책을 승인할 전국인민대표대회(전인대) 상무위원회가 다음...,이데일리,2024-11-02 17:49:11,https://n.news.naver.com/mnews/article/018/000...
4,'회계 투명성' 정책 강조하는 김병환號 금융위,금융위원회가 회계 정책과 관련 일정을 강화하는 모습이다. 임관 후 첫 업무를 회계 ...,아시아경제,2024-11-02 16:28:07,https://n.news.naver.com/mnews/article/277/000...
...,...,...,...,...,...
61,[속보]'고용쇼크' 무시한 월스트리트…나스닥 0.8%↑,"뉴욕증시가 하루 만에 일제히 상승했다. 10월 ‘고용쇼크’ 보고서가 나왔지만, 보잉...",이데일리,2024-11-02 05:12:14,https://n.news.naver.com/mnews/article/018/000...
62,전 세계가 들썩였는데…중국 증시는 '트럼프 트레이드' 없었다,중국은 가깝고도 먼 나라입니다. 서로를 의식하며 경쟁하고 때로는 의존하는 관계가 수...,머니투데이,2024-11-02 05:04:00,https://n.news.naver.com/mnews/article/008/000...
63,"부양책 쏟아내는 중국, 이번엔 다르다는데…""中주식 지금이 매수 타이밍""",중국 증시가 강력하고 지속적인 부양책으로 인해 강세를 나타낼 것이란 전망이 나온다....,머니투데이,2024-11-02 03:30:00,https://n.news.naver.com/mnews/article/008/000...
64,"뉴욕증시, 아마존 실적에 고무·고용 지표 충격 소화…반등 출발","김 현 연합인포맥스 통신원 = 뉴욕증시는 11월의 첫 거래일, 금주 마지막 거래일을...",연합뉴스,2024-11-02 00:07:34,https://n.news.naver.com/mnews/article/001/001...


In [9]:
# ------------
# csv 파일로 저장
# ------------
df.to_csv(f'data/증권뉴스_{date}.csv')

In [10]:
# ------------
# excel 파일로 저장
# ------------
df.to_excel(f'data/증권뉴스_{date}.xlsx')